# 1 . Import libraries

In [ ]:
import gdown
import pandas as pd

# 2 . Load data

In [ ]:
# Dữ liệu giao dịch (downloading from Google Drive to local path)
file_id = '104tCgXR04q30NzZhUarMAXaqIO5IAEKi'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'transactions_data.csv'
gdown.download(url, output, quiet=False)
trans = pd.read_csv(output)

# Thông tin cá nhân người dùng
file_id = '1lKEAkm4Yqo3sUeRooKqmMg7iR_LyhsW8'
url = 'https://drive.google.com/uc?id='
users = pd.read_csv(url + file_id)

# Thông tin thẻ
file_id = '1ZK-s7Ja6BgOCTfAygQYuGSZj4GrANFoR'
url = 'https://drive.google.com/uc?id='
cards = pd.read_csv(url + file_id)

# Thông tin mã ngành
file_id = '1E6fMv4QwbnubYz5l3b6PdNtMFy4S_Obq'
url = 'https://drive.google.com/uc?id='
mcc_raw = pd.read_json(url + file_id, typ='series')
mcc_codes = mcc_raw.reset_index()
mcc_codes.columns = ['mcc', 'description']

Downloading...
From (original): https://drive.google.com/uc?id=104tCgXR04q30NzZhUarMAXaqIO5IAEKi
From (redirected): https://drive.google.com/uc?id=104tCgXR04q30NzZhUarMAXaqIO5IAEKi&confirm=t&uuid=eb68b6cb-972e-46d0-beb7-276057037cf5
To: /content/transactions_data.csv
100%|██████████| 1.26G/1.26G [00:32<00:00, 38.7MB/s]


# 3 . Data clean

In [ ]:
trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13305915 entries, 0 to 13305914
Data columns (total 12 columns):
 #   Column          Dtype         
---  ------          -----         
 0   id              int64         
 1   date            datetime64[ns]
 2   client_id       int64         
 3   card_id         int64         
 4   amount          float64       
 5   use_chip        category      
 6   merchant_id     Int64         
 7   merchant_city   category      
 8   merchant_state  category      
 9   zip             Int64         
 10  mcc             Int64         
 11  errors          category      
dtypes: Int64(3), category(4), datetime64[ns](1), float64(1), int64(3)
memory usage: 926.7 MB


## 3.1. Fixing Transaction Data

In [ ]:
# Chuyển cột 'date' sang kiểu datetime
trans['date'] = pd.to_datetime(trans['date'])

# Loại bỏ ký tự $ trong cột 'amount', chuyển sang kiểu float
trans['amount'] = trans['amount'].replace('[\$,]', '', regex=True).astype(float)

# Convert negative values in 'amount' column into positive one
trans['amount'] = trans['amount'].abs()

# Chuyển cột 'merchant_id', 'zip', 'mcc' sang kiểu Integer
trans['merchant_id'] = trans['merchant_id'].astype('Int64')
trans['zip'] = trans['zip'].astype('Int64')
trans['mcc'] = trans['mcc'].astype('Int64')

# Loại bỏ chữ 'Transaction' trong cột 'use_chip', chuyển sang kiểu category
trans['use_chip'] = trans['use_chip'].str.replace(' Transaction', '', regex=False) #.str.strip()
trans['use_chip'] = trans['use_chip'].astype('category')

# Chuyển cột 'merchant_city', 'merchant_state', 'errors' sang kiểu category
trans['merchant_city'] = trans['merchant_city'].astype('category')
trans['merchant_state'] = trans['merchant_state'].astype('category')
trans['errors'] = trans['errors'].astype('category')

In [ ]:
trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13305915 entries, 0 to 13305914
Data columns (total 12 columns):
 #   Column          Dtype         
---  ------          -----         
 0   id              int64         
 1   date            datetime64[ns]
 2   client_id       int64         
 3   card_id         int64         
 4   amount          float64       
 5   use_chip        category      
 6   merchant_id     Int64         
 7   merchant_city   category      
 8   merchant_state  category      
 9   zip             Int64         
 10  mcc             Int64         
 11  errors          category      
dtypes: Int64(3), category(4), datetime64[ns](1), float64(1), int64(3)
memory usage: 926.7 MB


In [ ]:
# Checking null values
trans.isnull().sum()

,0
id,0
date,0
client_id,0
card_id,0
amount,0
use_chip,0
merchant_id,0
merchant_city,0
merchant_state,1563700
zip,1652706


In [ ]:
# Explaination of the null values

# The 'merchant_state' column has 1,563,700 null values which means all online transactions
# The 'zip' column has 1,652,706 null values which means the online transactions or the locations without zipcode
# The 'errors' column has 13,094,522 null values which means the successful transactions (without errors)

trans[trans['zip'].isnull()]

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
7,7475335,2010-01-01 00:14:00,1684,2140,26.46,Online,39021,ONLINE,NaN,<NA>,4784,NaN
8,7475336,2010-01-01 00:21:00,335,5131,261.58,Online,50292,ONLINE,NaN,<NA>,7801,NaN
18,7475346,2010-01-01 00:34:00,394,4717,26.04,Online,39021,ONLINE,NaN,<NA>,4784,NaN
24,7475353,2010-01-01 00:43:00,301,3742,10.17,Online,39021,ONLINE,NaN,<NA>,4784,NaN
26,7475356,2010-01-01 00:45:00,566,3439,16.86,Online,16798,ONLINE,NaN,<NA>,4121,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
13305879,23761832,2019-10-31 23:22:00,1556,2972,17.65,Online,88459,ONLINE,NaN,<NA>,5311,NaN
13305880,23761833,2019-10-31 23:22:00,1797,5660,34.81,Online,15143,ONLINE,NaN,<NA>,4784,NaN
13305888,23761843,2019-10-31 23:33:00,1069,5167,59.71,Online,39021,ONLINE,NaN,<NA>,4784,NaN
13305897,23761853,2019-10-31 23:39:00,1422,5696,694.30,Online,70268,ONLINE,NaN,<NA>,4722,NaN


In [ ]:
trans.sample(2)

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
9767111,19391608,2017-04-19 11:57:00,609,2892,77.03,Chip,11468,Garland,TX,75042,5970,NaN
299878,7833908,2010-04-01 12:59:00,1145,5444,19.08,Online,16798,ONLINE,NaN,<NA>,4121,NaN


## 3.2. Fixing Users Data

In [ ]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 2000 non-null   int64  
 1   current_age        2000 non-null   int64  
 2   retirement_age     2000 non-null   int64  
 3   birth_year         2000 non-null   int64  
 4   birth_month        2000 non-null   int64  
 5   gender             2000 non-null   object 
 6   address            2000 non-null   object 
 7   latitude           2000 non-null   float64
 8   longitude          2000 non-null   float64
 9   per_capita_income  2000 non-null   object 
 10  yearly_income      2000 non-null   object 
 11  total_debt         2000 non-null   object 
 12  credit_score       2000 non-null   int64  
 13  num_credit_cards   2000 non-null   int64  
dtypes: float64(2), int64(7), object(5)
memory usage: 218.9+ KB


In [ ]:
users.sample(2)

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
346,445,42,59,1977,12,Female,9864 Fifth Lane,43.88,-85.80,"$6,201","$12,643","$19,299",645,1
270,934,32,63,1987,12,Female,2017 Ninth Drive,42.38,-76.39,"$20,159","$41,106","$106,963",762,3


In [ ]:
# Chuyển cột 'gender' sang kiểu category
users['gender'] = users['gender'].astype('category')

# Chuyển cột 'address' sang kiểu string
users['address'] = users['address'].astype('string')

# Loại bỏ ký tự $ trong cột 'per_capita_income', 'yearly_income', 'total_debt', chuyển sang kiểu float
users['per_capita_income'] = users['per_capita_income'].replace('[\$,]', '', regex=True).astype(float)
users['yearly_income'] = users['yearly_income'].replace('[\$,]', '', regex=True).astype(float)
users['total_debt'] = users['total_debt'].replace('[\$,]', '', regex=True).astype(float)

In [ ]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   id                 2000 non-null   int64   
 1   current_age        2000 non-null   int64   
 2   retirement_age     2000 non-null   int64   
 3   birth_year         2000 non-null   int64   
 4   birth_month        2000 non-null   int64   
 5   gender             2000 non-null   category
 6   address            2000 non-null   string  
 7   latitude           2000 non-null   float64 
 8   longitude          2000 non-null   float64 
 9   per_capita_income  2000 non-null   float64 
 10  yearly_income      2000 non-null   float64 
 11  total_debt         2000 non-null   float64 
 12  credit_score       2000 non-null   int64   
 13  num_credit_cards   2000 non-null   int64   
dtypes: category(1), float64(5), int64(7), string(1)
memory usage: 205.3 KB


In [ ]:
users.sample(2)

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
1817,555,31,63,1988,5,Male,6692 Lake Street,38.03,-85.34,21444.0,43724.0,53853.0,514,1
772,1795,25,66,1994,10,Female,79 Hill Avenue,45.25,-93.33,29580.0,60310.0,91017.0,647,6


## 3.3. Fixing Cards Data

In [ ]:
cards.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6146 entries, 0 to 6145
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     6146 non-null   int64  
 1   client_id              6146 non-null   int64  
 2   card_brand             6146 non-null   object 
 3   card_type              6146 non-null   object 
 4   card_number            6146 non-null   float64
 5   expires                6146 non-null   object 
 6   cvv                    6146 non-null   int64  
 7   has_chip               6146 non-null   object 
 8   num_cards_issued       6146 non-null   int64  
 9   credit_limit           6146 non-null   object 
 10  acct_open_date         6146 non-null   object 
 11  year_pin_last_changed  6146 non-null   int64  
 12  card_on_dark_web       6146 non-null   object 
dtypes: float64(1), int64(5), object(7)
memory usage: 624.3+ KB


In [ ]:
# Chuyển cột 'card_brand', 'has_chip', 'card_on_dark_web' sang kiểu category
cards['card_brand'] = cards['card_brand'].astype('category')
cards['has_chip'] = cards['has_chip'].astype('category')
cards['card_on_dark_web'] = cards['card_on_dark_web'].astype('category')

# Loại bỏ cụm từ 'Debit ( )' trong cột 'card_type', chuyển sang kiểu category
cards['card_type'] = cards['card_type'].str.replace('Debit (', '', regex=False) #.str.strip()
cards['card_type'] = cards['card_type'].str.replace(')', '', regex=False) #.str.strip()
cards['card_type'] = cards['card_type'].astype('category')

# Chuyển cột 'card_number' sang kiểu integer
cards['card_number'] = cards['card_number'].astype('Int64')

# Chuyển cột 'expires', 'acct_open_date' sang kiểu datetime
cards['expires'] = pd.to_datetime(cards['expires'], format='%b-%y')
cards['acct_open_date'] = pd.to_datetime(cards['acct_open_date'], format='%b-%y')

# Loại bỏ ký tự $ trong cột 'credit_limit', chuyển sang kiểu float
cards['credit_limit'] = cards['credit_limit'].replace('[\$,]', '', regex=True).astype(float)

In [ ]:
cards.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6146 entries, 0 to 6145
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     6146 non-null   int64         
 1   client_id              6146 non-null   int64         
 2   card_brand             6146 non-null   category      
 3   card_type              6146 non-null   category      
 4   card_number            6146 non-null   Int64         
 5   expires                6146 non-null   datetime64[ns]
 6   cvv                    6146 non-null   int64         
 7   has_chip               6146 non-null   category      
 8   num_cards_issued       6146 non-null   int64         
 9   credit_limit           6146 non-null   float64       
 10  acct_open_date         6146 non-null   datetime64[ns]
 11  year_pin_last_changed  6146 non-null   int64         
 12  card_on_dark_web       6146 non-null   category      
dtypes: 

In [ ]:
cards.sample(2)

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
633,4321,891,Mastercard,Debit,5895478812166500,2024-01-01,598,YES,2,8212.0,2010-08-01,2011,No
2219,5215,1379,Visa,Debit,4808036256579380,2017-11-01,592,YES,2,12460.0,2010-10-01,2010,No


## 3.4. Merchant Category Codes data

The list is Standard classification codes for business types without any duplicating or missing values.

In [ ]:
# checking duplicated values
mcc_codes.duplicated().sum()

np.int64(0)

In [ ]:
# checking null values
mcc_codes.isnull().sum()

,0
mcc,0
description,0


In [ ]:
mcc_codes.sample(2)

,mcc,description
64,3771,Railroad Passenger Transport
9,5311,Department Stores


# 4 . EDA

## 4.1. What is the Lifetime Value (LTV) of each customer group?

In [ ]:
ltv = trans.groupby(['client_id', 'date'])['amount'].sum().reset_index()
# ltv = trans.groupby('client_id')['am#ount'].sum()
ltv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13219873 entries, 0 to 13219872
Data columns (total 3 columns):
 #   Column     Dtype         
---  ------     -----         
 0   client_id  int64         
 1   date       datetime64[ns]
 2   amount     float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 302.6 MB


In [ ]:
ltv.head()

,client_id,date,amount
1496059,218,2018-10-13 11:34:00,6.74
5764095,924,2016-07-23 05:09:00,-164.00
